*Tester usado: DT380B*

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO%201/img/error_DT380B.png?raw=true:, width=100" alt="My Image" width=500>

https://www.viaindustrial.com/manuales_pdf/multimetro-digital-economico-basico-96830-dt830-b-ebchq-manual-espanol.pdf

**Patron HP 3468b **

https://xdevs.com/doc/HP_Agilent_Keysight/HP%203468A,%20B%20Operating.pdf

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO%201/img/error_HP3468B.png?raw=true:, width=100" alt="My Image" width=500>


In [1]:
import statistics as st
import numpy as np
import math as mt

mA = 0.001
uA = 0.000001

res = [1e-6,1e-5,1e-4]

def incertidumbre_A(X) -> float:
  return st.stdev(X)/mt.sqrt(len(X))

#Esta funcion es para error en cuentas. Si nos dan el error
#fijo, usamos la formula, adaptando la manera en la que le enviamos los argumentos
def inecrtidumbre_B(er_p, nd, Ci, Xm) -> float:
  return Xm*((er_p/100)+(nd/Ci))/mt.sqrt(3)

def incertidumbre_total_A_B(uA, uB) -> float:
  return mt.sqrt( uA**2 + uB**2 )

def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return mt.trunc(number)

    factor = 10.0 ** decimals
    return mt.trunc(number * factor) / factor

# Procedimiento específico de la medición:
Utilizamos una fuente generadora de corriente, a la cual se le dieron distintos valores de corriente para vertificar. En nuestro caso generaron corrientes de 100uA, 1.5mA y 6.75mA.

Cada una de estas corrientes fue medida usando 2 amperimetros en serie. Uno era el amperimetro patron y el otro era el generico. A su vez por cada valor de corriente se usaron 2 escalas distintas en el amperimetro generico, y por cada escala se tomaron 5 valores de cada amperimetro.

A partir de las valores obtenidos se calculan las incertidumbres, y se determina si se verifica la calibracion.

#**Escala: 2000uA**
#**Fuente: 100uA**

In [2]:
V_ver = [0.00010,0.00010,0.00010,0.00009,0.00009]
V_med = [114,115,115,114,115]#uA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*uA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m*uA}\necal_m: {ecal_m}')

V_ver_m: 9.6e-05
V_med_m: 0.0001146
ecal_m: 1.859999999999999e-05


##Incertidumbres de tipo A

In [3]:
V_med_uA = list(map(lambda x: x*uA, V_med))#paso de uA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_uA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831775e-06
uA_V_med: 2.449489742783171e-07


##Incertidumbres de tipo B


Al calcular las cuentas ci para ambas mediciones, vemos que en **este caso en particular** todas las medidiones obtenidas con nuestro tester son de 3 digitos no decimales. Por eso no es necesario multiplicar por ningun numero para obtener las ci. Para el valor medio patron, como tiene 5 decimales lo multiplicamos por 100000,y al hacer int() los numeros decimamles se truncan. Asi obtenemos las ci.

**(el mismo proceso se repetira para las otras mediciones)**

In [22]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*1) #son 3 digitos, y V_med_m esta en uA sin decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[0]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 4.1290564481730184e-05
uB_V_med: 2.886751345948129e-07


##Incertidumbre total de cada magnitud


In [23]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

Incertidumbre total
uc_V_ver: 4.1290564481730184e-05
uc_V_med: 2.000208322483757e-05


##Incertidumbre combinada - funcion error de calculo

In [24]:
uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

uc_ecal: 4.588021412933081e-05


##Incertidumbre expandida

In [25]:
k95 = 2

Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

Uc_ecal: 9.176042825866162e-05


## verificacion calibracion

Para que la calibracion sea aceptabe, $|e_{cal_m}|+Uc_{ecal} <= \Delta V$

In [26]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*uA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 9.176042825866162e-05
delta_V: 2.3278e-06
|ecal_m|+Uc_ecal = 9.176045425866163e-05
NO verifica calibracion


#**Escala: 20mA**
#**Fuente: 100uA**

In [27]:
V_ver = [0.00010,0.00010,0.00010,0.00009,0.00009]
V_med = [0.12,0.12,0.11,0.11,0.12]#mA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*mA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 9.6e-05
V_med_m: 0.11599999999999999
ecal_m: 1.9999999999999998e-05


##Incertidumbres de tipo A

In [28]:
V_med_mA = list(map(lambda x: x*mA, V_med))#paso de mA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_mA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831775e-06
uA_V_med: 2.4494897427831775e-06


##Incertidumbres de tipo B

In [29]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*100) #son 3 digitos, y V_med_m esta con 2 decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[1]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 3.70446407920678e-05
uB_V_med: 2.8867513459481293e-06


##Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [30]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 3.70446407920678e-05
uc_V_med: 3.7859388972001827e-06
uc_ecal: 3.723759853624649e-05


##verificacion calibracion

In [31]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*mA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 7.447519707249298e-05
delta_V: 2.2250909090909094e-05
|ecal_m|+Uc_ecal = 7.447521707249298e-05
NO verifica calibracion


#**Escala: 20mA**
#**Fuente: 1,5 mA**

In [32]:
V_ver = [0.00149,0.00149,0.00148,0.00148,0.00149]
V_med = [1.50,1.51,1.51,1.50,1.51]#mA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*mA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 0.001486
V_med_m: 1.506
ecal_m: 2.0000000000000052e-05


##Incertidumbres de tipo A

In [33]:
V_med_mA = list(map(lambda x: x*mA, V_med))#paso de mA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_mA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831843e-06
uA_V_med: 2.4494897427831843e-06


##Incertidumbres de tipo B

In [34]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*1000) #son 4 digitos, y V_med_m esta con 3 decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[1]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 3.623995495336942e-05
uB_V_med: 2.8867513459481293e-06


##Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [35]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 3.623995495336942e-05
uc_V_med: 3.785938897200187e-06
uc_ecal: 3.6437174264143734e-05


##verificacion calibracion

In [36]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*mA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 7.287434852828747e-05
delta_V: 1.706e-05
|ecal_m|+Uc_ecal = 7.287436852828747e-05
NO verifica calibracion


#**Escala: 2000uA**
#**Fuente: 1,5 mA**

In [37]:
V_ver = [0.00144,0.00144,0.00143,0.00143,0.00144]
V_med = [1462,1461,1462,1462,1461]#uA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*uA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 0.001436
V_med_m: 1461.6
ecal_m: 2.5599999999999755e-05


##Incertidumbres de tipo A

In [38]:
V_med_uA = list(map(lambda x: x*uA, V_med))#paso de mA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_uA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831843e-06
uA_V_med: 2.449489742783503e-07


##Incertidumbres de tipo B

In [39]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*1) #son 4 digitos, y V_med_m esta en uA sin decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[0]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 3.6195790549438136e-05
uB_V_med: 2.886751345948129e-07


##Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [40]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 3.6195790549438136e-05
uc_V_med: 3.785938897200393e-07
uc_ecal: 3.6197770467697704e-05


##verificacion calibracion

In [41]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*uA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 7.239554093539541e-05
delta_V: 1.661682135523614e-05
|ecal_m|+Uc_ecal = 7.23955665353954e-05
NO verifica calibracion


#**Escala: 20mA**
#**Fuente: 6,75mA**

In [42]:
V_ver = [0.00674,0.00674,0.00673,0.00673,0.00674]
V_med = [6.76,6.76,6.77,6.77,6.76]#mA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*mA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 0.006736
V_med_m: 6.763999999999999
ecal_m: 2.799999999999938e-05


##Incertidumbres de tipo A

In [43]:
V_med_mA = list(map(lambda x: x*mA, V_med))#paso de mA a Ampere
print(V_med_mA)
print(V_ver)
uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_mA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

[0.0067599999999999995, 0.0067599999999999995, 0.00677, 0.00677, 0.0067599999999999995]
[0.00674, 0.00674, 0.00673, 0.00673, 0.00674]
Incertidumbre tipo A
uA_V_ver: 2.4494897427832906e-06
uA_V_med: 2.4494897427832906e-06


##Incertidumbres de tipo B

In [44]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*100) #son 3 digitos, y V_med_m esta con 2 decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[1]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 4.128325307500771e-05
uB_V_med: 2.8867513459481293e-06


##Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [45]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 4.128325307500771e-05
uc_V_med: 3.7859388972002555e-06
uc_ecal: 4.145648704109488e-05


##verificacion calibracion

In [46]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*mA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*mA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 8.291297408218975e-05
delta_V: 8.765183431952661e-05
|ecal_m|+Uc_ecal = 8.294097408218975e-05
Verifica calibracion


#**Escala: 200mA**
#**Fuente: 6,75mA**

In [15]:
V_ver = [0.00675,0.00675,0.00676,0.00676,0.00675]
V_med = [6.8,6.8,6.7,6.8,6.8]#mA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*mA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 0.0067540000000000005
V_med_m: 6.78
ecal_m: 2.599999999999998e-05


##Incertidumbres de tipo A

In [16]:
V_med_mA = list(map(lambda x: x*mA, V_med))#paso de mA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_mA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427832906e-06
uA_V_med: 1.999999999999988e-05


##Incertidumbres de tipo B

In [21]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 5
er_p_med = 1.2
Ci_V_med = int(V_med_m*10) #son 2 digitos, y V_med_m esta con 1 decimal

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[2]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 4.1290564481730184e-05
uB_V_med: 2.8867513459481293e-05


##Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [20]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 4.1290564481730184e-05
uc_V_med: 3.5118845842842394e-05
uc_ecal: 5.420557211720255e-05


##verificacion calibracion

In [14]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*mA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*mA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m' + 'Verifica calibracion')
else: print('\033[1m' + 'NO verifica calibracion')

Uc_ecal: 0.0001084111442344051
delta_V: 0.0005873301492537313
|ecal_m|+Uc_ecal = 0.0001084371442344051
Verifica calibracion
